# Nanodegree Engenheiro de Machine Learning
## Aprendizagem Supervisionada
## Projeto 2: Construindo um Sistema de Intervenção para Estudantes

Bem-vindo ao segundo projeto do Nanodegree de Machine Learning! Neste Notebook, alguns templates de código já foram fornecidos, e será o seu trabalho implementar funcionalidades necessárias para completar este projeto com êxito. Seções que começam com **'Implementação'** no cabeçalho indicam que o bloco de código que se segue precisará de funcionalidades adicionais que você deve fornecer. Instruções serão providenciadas para cada seção e as especificações para cada implementação estarão marcadas no bloco de código com o comando `'TODO'`. Tenha certeza de ler atentamente todas as instruções!

Além do código implementado, haverá questões relacionadas ao projeto e à implementação que você deve responder. Cada seção em que você tem que responder uma questão será antecedida de um cabeçalho **'Questão X'**. Leia atentamente cada questão e escreva respostas completas nas caixas de texto subsequentes que começam com **'Resposta: '**. O projeto enviado será avaliado baseado nas respostas para cada questão e a implementação que você forneceu.  

>**Nota:** Células de código e Markdown podem ser executadas utilizando o atalho de teclado **Shift + Enter**. Além disso, as células Markdown podem ser editadas, um clique duplo na célula entra no modo de edição.

### Questão 1 - Classificação versus Regressão
*Seu objetivo neste projeto é identificar estudantes que possam precisar de intervenção antecipada antes de serem reprovados. Que tipo de problema de aprendizagem supervisionada é esse: classificação ou regressão? Por quê?*

**Resposta: **

Trata-se um problema de classificação - o objetivo é classificar cada um dos estudantes em um dentre dois grupos: os que precisam de algum tipo de intervenção, e os que não precisam.

## Observando os Dados
Execute a célula de código abaixo para carregar as bibliotecas de Python necessárias e os dados sobre os estudantes. Note que a última coluna desse conjunto de dados, `'passed'`, será nosso rótulo alvo (se o aluno foi ou não aprovado). As outras colunas são atributos sobre cada aluno.

In [1]:
# Importar bibliotecas
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

In [2]:
# Ler os dados dos estudantes
student_data = pd.read_csv("student-data.csv")
print "Os dados dos estudantes foram lidos com êxito!"

Os dados dos estudantes foram lidos com êxito!


### Implementação: Observando os Dados
Vamos começar observando o conjunto de dados para determinar quantos são os estudantes sobre os quais temos informações e entender a taxa de graduação entre esses estudantes. Na célula de código abaixo, você vai precisar calcular o seguinte:
- O número total de estudantes, `n_students`.
- O número total de atributos para cada estudante, `n_features`.
- O número de estudantes aprovados, `n_passed`.
- O número de estudantes reprovados, `n_failed`.
- A taxa de graduação da classe, `grad_rate`, em porcentagem (%).


In [3]:
# Importa o comportamento de divisao do Python 3 (divisão de inteiros resulta em float)
from __future__ import division

# Calcule o número de estudantes
n_students = student_data.shape[0]

# Calcule o número de atributos
n_features = student_data.shape[1]-1 

# Calcule o número de alunos aprovados
n_passed = student_data["passed"].value_counts()["yes"]

# Calcule o número de alunos reprovados
n_failed = student_data["passed"].value_counts()["no"]

# Calcule a taxa de graduação
grad_rate = n_passed/n_students*100

# Imprima os resultados
print "Número total de estudantes: {}".format(n_students)
print "Número de atributos: {}".format(n_features)
print "Número de estudantes aprovados: {}".format(n_passed)
print "Número de estudantes reprovados: {}".format(n_failed)
print "Taxa de graduação: {:.2f}%".format(grad_rate)

Número total de estudantes: 395
Número de atributos: 30
Número de estudantes aprovados: 265
Número de estudantes reprovados: 130
Taxa de graduação: 67.09%


## Preparando os Dados
Nesta seção, vamos preparara os dados para modelagem, treinamento e teste.

### Identificar atributos e variáveis-alvo
É comum que os dados que você obteve contenham atributos não numéricos. Isso pode ser um problema, dado que a maioria dos algoritmos de machine learning esperam dados númericos para operar cálculos.

Execute a célula de código abaixo para separar os dados dos estudantes em atributos e variáveis-alvo e verificar se algum desses atributos é não numérico.

In [4]:
# Extraia as colunas dos atributos
feature_cols = list(student_data.columns[:-1])

# Extraia a coluna-alvo 'passed'
target_col = student_data.columns[-1] 

# Mostre a lista de colunas
print "Colunas de atributos:\n{}".format(feature_cols)
print "\nColuna-alvo: {}".format(target_col)

# Separe os dados em atributos e variáveis-alvo (X_all e y_all, respectivamente)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Mostre os atributos imprimindo as cinco primeiras linhas
print "\nFeature values:"
print X_all.head()

Colunas de atributos:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Coluna-alvo: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Pré-processar Colunas de Atributo

Como você pode ver, há muitas colunas não numéricas que precisam ser convertidas! Muitas delas são simplesmente `yes`/`no`, por exemplo, a coluna `internet`. É razoável converter essas variáveis em valores (binários) `1`/`0`.

Outras colunas, como `Mjob` e `Fjob`, têm mais do que dois valores e são conhecidas como variáveis categóricas. A maneira recomendada de lidar com esse tipo de coluna é criar uma quantidade de colunas proporcional aos possíveis valores (por exemplo, `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc), e assinalar `1` para um deles e `0` para todos os outros.

Essas colunas geradas são por vezes chamadas de _variáveis postiças_ (_dummy variables_), e nós iremos utilizar a função [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) para fazer essa conversão. Execute a célula de código abaixo para executar a rotina de pré-processamento discutida nesta seção.

In [5]:
def preprocess_features(X):
    ''' Pré-processa os dados dos estudantes e converte as variáveis binárias não numéricas em
        variáveis binárias (0/1). Converte variáveis categóricas em variáveis postiças. '''
    
    # Inicialize nova saída DataFrame
    output = pd.DataFrame(index = X.index)

    # Observe os dados em cada coluna de atributos 
    for col, col_data in X.iteritems():
        
        # Se o tipo de dado for não numérico, substitua todos os valores yes/no por 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # Se o tipo de dado for categórico, converta-o para uma variável dummy
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Reúna as colunas revisadas
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementação: Divisão dos Dados de Treinamento e Teste
Até agora, nós convertemos todos os atributos _categóricos_ em valores numéricos. Para o próximo passo, vamos dividir os dados (tanto atributos como os rótulos correspondentes) em conjuntos de treinamento e teste. Na célula de código abaixo, você irá precisar implementar o seguinte:
- Embaralhe aleatoriamente os dados (`X_all`, `y_all`) em subconjuntos de treinamento e teste.
  - Utilizar 300 pontos de treinamento (aproxidamente 75%) e 95 pontos de teste (aproximadamente 25%).
  - Estabelecer um `random_state` para as funções que você utiliza, se a opção existir.
  - Armazene os resultados em `X_train`, `X_test`, `y_train` e `y_test`.

In [6]:
# Importa módulo de randomização e separação de conjuntos de treino e teste
from sklearn.model_selection import train_test_split 

# Estabeleça o número de pontos de treinamento
NUM_TRAIN = 300

# Estabeleça o número de pontos de teste
NUM_TEST = X_all.shape[0] - NUM_TRAIN

# Emabaralhe e distribua o conjunto de dados de acordo com o número de pontos de treinamento e teste abaixo
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=NUM_TEST, shuffle=True, random_state=42)

# Mostre o resultado da distribuição
print "O conjunto de treinamento tem {} amostras.".format(X_train.shape[0])
print "O conjunto de teste tem {} amostras.".format(X_test.shape[0])

O conjunto de treinamento tem 300 amostras.
O conjunto de teste tem 95 amostras.


## Treinando e Avaliando Modelos
Nesta seção, você irá escolher 3 modelos de aprendizagem supervisionada que sejam apropriados para esse problema e que estejam disponíveis no `scikit-learn`. Primeiro você irá discutir o raciocínio por trás da escolha desses três modelos considerando suas vantagens e desvantagens e o que você sabe sobre os dados. Depois você irá ajustar o modelo a diferentes tamanhos de conjuntos de treinamento (com 100, 200 e 300 pontos) e medir a pontuação F<sub>1</sub>. Você vai precisar preencher três tabelas (uma para cada modelo) que mostrem o tamanho do conjunto de treinamento, o tempo de treinamento, o tempo de previsão e a pontuação F<sub>1</sub> no conjunto de treinamento.

**Os seguintes modelos de aprendizagem supervisionada estão atualmente disponíveis no **[`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html)** para você escolher:**
- Gaussian Naive Bayes (GaussianNB)
- Árvores de Decisão
- Métodos de agregação (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Método do gradiente estocástico (SGDC)
- Máquinas de vetores de suporte (SVM)
- Regressão logística

### Questão 2 - Aplicação dos Modelos
*Liste três modelos de aprendizagem supervisionada que são apropriadas para esse problema. Para cada modelo escolhido:*
- Descreva uma aplicação em mundo real na indústria em que o modelo pode ser aplicado. *(Talvez você precise fazer um pouco de pesquisa para responder essa questão – dê as devidas referências!)* 
- Quais são as vantagens do modelo; quando ele tem desempenho melhor? 
- Quais são as desvantagens do modelo, quando ele tem desempenho pior?
- O que faz desse modelo um bom candidato para o problema, considerando o que você sabe sobre os dados?

**Resposta: **

Serão utilizados os modelos *Logistic Regression, Support Vector Machines (SVMs) e K-Nearest Neighbors*.

**Logistic Regression**

Este classificador foi escolhido por conta de sua simplicidade e por ser um modelo apropriado para dados discretos, e para classificar respostas binárias. Uma desvantagem deste modelo é presumir que as variáveis sejam independentes, o que pode não ser verdade para todas as variáveis para este conjunto de dados. 

Este modelo é utilizado para diversas aplicações como, por exemplo, predição de risco de desenvolvimento de doenças, predição de falha de componentes de engenharia, predição da chance de um consumidor adquirir um bem ou serviço (https://en.wikipedia.org/wiki/Logistic_regression#Fields_and_example_applications).

O modelo foi escolhido devido à característica binária da variável resposta, e por conta do conjunto de dados ser composto de um número relativamente grande de variáveis categóricas.

**Support Vector Machine**

Este modelo é apropriado para classificação e para regressão de dados multivariados, sendo capaz de lidar também com dados contínuos. O modelo cria fronteiras na forma de hiperplanos, capazes de separar os dados em diferentes categorias, podendo ser uma classificação não-binária. Uma limitação é o fato de estas fronteiras - hiperplanos - serem lineares, sendo necessário fazer uso do *kernel trick* para contornar esta limitação, o que pode aumentar o esforço de ajuste do modelo.

SVM é aplicado com sucesso emciências biológicas para classificar proteínas e outros compostos. Além disso, ele também é aplicado em análise e categorização de texto e classificação de imagens (https://en.wikipedia.org/wiki/Support_vector_machine#Applications).

A escolha deste algoritmo aqui foi motivada também pela sua relativa simplicidade e eficiência em classificação de dados.

**K-Nearest Neighbors**

Este algoritmo classifica uma observação com base nos valores obtidos para os *K* vizinhos mais próximos. Devido a esta característica, o custo computacional para treinamento é relativamente baixo, porém o custo computacional para teste e predição acaba podendo tornar-se elevado. Este modelo também pode ser sensível a outliers, além de sofrer da *maldição da dimensionalidade*.

Este algoritmo é utilizado na classificação de documentos e diagnóstico de câncer (https://ieeexplore.ieee.org/document/7951888/, https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2243774/).

Este algoritmo foi escolhido diante da base de dados relativamente pequena e pouca exigência em termos de predição, de modo que o custo computacional para teste e predição não deveria se tornar um gargalo.

### Configuração
Execute a célula de código abaixo para inicializar três funções de ajuda que você pode utilizar para treinar e testar os três modelos de aprendizagem supervisionada que você escolheu acima. As funções são as seguintes:
- `train_classifier` - recebe como parâmetro um classificador e dados de treinamento e ajusta o classificador aos dados.
- `predict_labels` - recebe como parâmetro um classificador ajustado, atributos e rótulo alvo e faz estimativas utilizando a pontuação do F<sub>1</sub>.
- `train_predict` - recebe como entrada um classificador, e dados de treinamento e teste, e executa `train_clasifier` e `predict_labels`.
 - Essa função vai dar a pontuação F<sub>1</sub> tanto para os dados de treinamento como para os de teste, separadamente.

In [7]:
def train_classifier(clf, X_train, y_train):
    ''' Ajusta um classificador para os dados de treinamento. '''
    
    # Inicia o relógio, treina o classificador e, então, para o relógio
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Imprime os resultados
    print "O modelo foi treinado em {:.4f} segundos".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Faz uma estimativa utilizando um classificador ajustado baseado na pontuação F1. '''
    
    # Inicia o relógio, faz estimativas e, então, o relógio para
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Imprime os resultados de retorno
    print "As previsões foram feitas em {:.4f} segundos.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Treina e faz estimativas utilizando um classificador baseado na pontuação do F1. '''
    
    # Indica o tamanho do classificador e do conjunto de treinamento
    print "Treinando um {} com {} pontos de treinamento. . .".format(clf.__class__.__name__, len(X_train))
    
    # Treina o classificador
    train_classifier(clf, X_train, y_train)
    
    # Imprime os resultados das estimativas de ambos treinamento e teste
    print "Pontuação F1 para o conjunto de treino: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "Pontuação F1 para o conjunto de teste: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementação: Métricas de Desempenho do Modelo
Com as funções acima, você vai importar os três modelos de aprendizagem supervisionada de sua escolha e executar a função `train_prediction` para cada um deles. Lembre-se de que você vai precisar treinar e usar cada classificador para três diferentes tamanhos de conjuntos de treinamentos: 100, 200 e 300 pontos. Então você deve ter 9 saídas diferentes abaixo – 3 para cada modelo utilizando cada tamanho de conjunto de treinamento. Na célula de código a seguir, você deve implementar o seguinte:
- Importe os três modelos de aprendizagem supervisionada que você escolheu na seção anterior.
- Inicialize os três modelos e armazene eles em `clf_A`, `clf_B` e `clf_C`.
 - Defina um `random_state` para cada modelo, se a opção existir.
 - **Nota:** Utilize as configurações padrão para cada modelo – você vai calibrar um modelo específico em uma seção posterior.
- Crie diferentes tamanhos de conjuntos de treinamento para treinar cada modelo.
 - *Não embaralhe e distribua novamente os dados! Os novos pontos de treinamento devem ser tirados de `X_train` e `y_train`.*
- Treine cada modelo com cada tamanho de conjunto de treinamento e faça estimativas com o conjunto de teste (9 vezes no total).  
**Nota:** Três tabelas são fornecidas depois da célula de código a seguir, nas quais você deve anotar seus resultados.

In [8]:
# Importe os três modelos de aprendizagem supervisionada do sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# Inicialize os três modelos
clf_A = LogisticRegression()
clf_B = SVC()
clf_C = KNeighborsClassifier()

# Configure os tamanho dos conjuntos de treinamento. 
# Os dados já foram randomizados, então será tomado um subset a partir do começo
X_train_100 = X_train.iloc[0:100]
y_train_100 = y_train.iloc[0:100]

X_train_200 = X_train.iloc[0:200]
y_train_200 = y_train.iloc[0:200]

X_train_300 = X_train
y_train_300 = y_train

classifiers = [clf_A, clf_B, clf_C]
X_train_sets = [X_train_100, X_train_200, X_train]
y_train_sets = [y_train_100, y_train_200, y_train]

# Executar a função 'train_predict' para cada classificador e cada tamanho de conjunto de treinamento
for clf in classifiers:
    for X_train, y_train in zip(X_train_sets, y_train_sets):
        train_predict(clf, X_train, y_train, X_test, y_test)
        print ""

Treinando um LogisticRegression com 100 pontos de treinamento. . .
O modelo foi treinado em 0.0240 segundos
As previsões foram feitas em 0.0120 segundos.
Pontuação F1 para o conjunto de treino: 0.8593.
As previsões foram feitas em 0.0000 segundos.
Pontuação F1 para o conjunto de teste: 0.7647.

Treinando um LogisticRegression com 200 pontos de treinamento. . .
O modelo foi treinado em 0.0040 segundos
As previsões foram feitas em 0.0030 segundos.
Pontuação F1 para o conjunto de treino: 0.8562.
As previsões foram feitas em 0.0000 segundos.
Pontuação F1 para o conjunto de teste: 0.7914.

Treinando um LogisticRegression com 300 pontos de treinamento. . .
O modelo foi treinado em 0.0040 segundos
As previsões foram feitas em 0.0000 segundos.
Pontuação F1 para o conjunto de treino: 0.8468.
As previsões foram feitas em 0.0000 segundos.
Pontuação F1 para o conjunto de teste: 0.8060.

Treinando um SVC com 100 pontos de treinamento. . .
O modelo foi treinado em 0.0040 segundos
As previsões foram 

### Resultados em tabelas
Edite a célula abaixo e veja como a tabela pode ser desenhada em [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). Você deve salvar seus resultados abaixo nas tabelas fornecidas.

** Classificador 1 - LogisticRegression **  

| Tamanho do Conjunto de Treinamento | Tempo de Treinamento (s) | Tempo de Estimativa (teste) (s) | Pontuação F1 (treinamento) | Pontuação F1 (teste) |
| :--------------------------------: | :------------------: | :-------------------------: | :------------------------: | :------------------: |
| 100 | 0.0015 | 0.0002 | 0.8593 | 0.7647 |
| 200 | 0.0022 | 0.0002 | 0.8562 | 0.7914 |
| 300 | 0.0028 | 0.0002 | 0.8468 | 0.8060 |

** Classificador 2 - SVC**  

| Tamanho do Conjunto de Treinamento | Tempo de Treinamento (s)| Tempo de Estimativa (teste) (s)| Pontuação F1 (treinamento) | Pontuação F1 (teste) |
| :--------------------------------: | :------------------: | :-------------------------: | :------------------------: | :------------------: |
| 100   |0.0021 |0.0012 |0.8777 |0.7746 |
| 200   |0.0047 |0.0017 | 0.8679|0.7815 |
| 300   |0.0100 | 0.0025  |0.8761 |0.7838 |

** Classificador 3 - KNeighborsClassifier**  

| Tamanho do Conjunto de Treinamento | Tempo de Treinamento (s)| Tempo de Estimativa (teste) (s)| Pontuação F1 (treinamento) | Pontuação F1 (teste) |
| :--------------------------------: | :------------------: | :-------------------------: | :------------------------: | :------------------: |
| 100 | 0.0010 | 0.0016 | 0.8060 | 0.7246 |
| 200 | 0.0010 | 0.0021 | 0.8800 | 0.7692 |
| 300 | 0.0012 | 0.0027 | 0.8809 | 0.7801 |

## Escolhendo o Melhor Modelo
Nesta seção final, você irá escolher dos três modelos de aprendizagem supervisionada o *melhor* para utilizar os dados dos estudantes. Você então executará um busca em matriz otimizada para o modelo em todo o conjunto de treinamento (`X_train` e `y_train`) ao calibrar pelo menos um parâmetro, melhorando em comparação a pontuação F<sub>1</sub> do modelo não calibrado. 

### Questão 3 - Escolhendo o Melhor Modelo
*Baseando-se nos experimentos que você executou até agora, explique em um ou dois parágrafos ao conselho de supervisores qual modelo que você escolheu como o melhor. Qual modelo é o mais apropriado baseado nos dados disponíveis, recursos limitados, custo e desempenho?*

**Resposta: **

O modelo mais apropriado foi o de *regressão logística (LogisticRegression)*. Este modelo apresentou um bom desempenho pelo score F1, tanto para treinamento quanto para teste. Os outros dois modelos apresentaram score F1 para treino batante elevado, porém os scores F1 de teste se mostraram baixos, o que é uma indicação de sobreajuste *(overfitting)*.

Além disso, de todos os modelos o modelo de *regressão logística* apresentou melhor desempenho computacional, com tempo de treinamento da mesma ordem de grandeza do tempo de teste para o modelo KNN. O modelo SVC apresentou os piores índices de desempenho computacional dentre os três.

### Questão 4 – O Modelo para um Leigo
*Em um ou dois parágrafos, explique para o conselho de supervisores, utilizando termos leigos, como o modelo final escolhido deve trabalhar. Tenha certeza que você esteja descrevendo as melhores qualidades do modelo, por exemplo, como o modelo é treinado e como ele faz uma estimativa. Evite jargões técnicos ou matemáticos, como descrever equações ou discutir a implementação do algoritmo.*

**Resposta: **

Na regressão linear, cada observação é utilizada para encontrar uma função matemática, uma curva, que correlaciona os valores das observações com os valores de uma variável alvo. A regressão logística funciona de forma semelhante, porém ao invés de uma variável alvo, a função nos retorna a _probabilidade_ que cada observação tem de pertencer ou não a uma classe. As observações são utilizadas para encontrar os coeficientes dessa função. Depois que tivermos os coeficientes, podemos prever qual a classe mais provável para uma nova observação.

Este modelo tem a qualidade de ser computacionalmente pouco exigente simples - ele utiliza uma função matemática predeterminada, a função logística, bastando somente encontrar alguns coeficientes para ela, inclusive em situações onde hajam várias características diferentes ao mesmo tempo (dados multidimensionais).

### Implementação: Calibrando o Modelo (_Tuning_)
Calibre o modelo escolhido. Utilize busca em matriz (`GridSearchCV`) com, pelo menos, um parâmetro importante calibrado com, pelo menos, 3 valores diferentes. Você vai precisar utilizar todo o conjunto de treinamento para isso. Na célula de código abaixo, você deve implementar o seguinte:
- Importe [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) e [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Crie o dicionário de parâmetros que você deseja calibrar para o modelo escolhido.
 - Examplo: `parameters = {'parameter' : [list of values]}`.
- Inicialize o classificador que você escolheu e armazene-o em `clf`.
- Crie a função de pontuação F<sub>1</sub> utilizando `make_scorer` e armazene-o em `f1_scorer`.
 - Estabeleça o parâmetro `pos_label` para o valor correto!
- Execute uma busca em matriz no classificador `clf` utilizando o `f1_scorer` como método de pontuação e armazene-o em `grid_obj`.
- Treine o objeto de busca em matriz com os dados de treinamento (`X_train`, `y_train`) e armazene-o em `grid_obj`.

In [9]:
# Importe 'GridSearchCV' e 'make_scorer'
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score
from sklearn.linear_model import LogisticRegression

# Crie a lista de parâmetros que você gostaria de calibrar
parameters = {
    'penalty' : ['l1', 'l2'],
    'C' : [0.01, 0.05, 0.1, 0.3, 0.5, 0.7, 1, 5, 10, 50, 100]
#    'C' : [0.5]
}

# Inicialize o classificador
clf = LogisticRegression()

# Faça uma função de pontuação f1 utilizando 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# Execute uma busca em matriz no classificador utilizando o f1_scorer como método de pontuação
grid_obj = GridSearchCV(clf, param_grid=parameters, scoring=f1_scorer)

# Ajuste o objeto de busca em matriz para o treinamento de dados e encontre os parâmetros ótimos
grid_obj = grid_obj.fit(X_train, y_train)

# Atribui ao classificador o melhor estimador obtido pela busca em matriz
clf = grid_obj.best_estimator_

# Reporte a pontuação final F1 para treinamento e teste depois de calibrar os parâmetrosprint "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "O modelo calibrado tem F1 de {:.4f} no conjunto de treinamento.".format(predict_labels(clf, X_train, y_train))
print "O modelo calibrado tem F1 de {:.4f} no conjunto de teste.".format(predict_labels(clf, X_test, y_test))


As previsões foram feitas em 0.0000 segundos.
O modelo calibrado tem F1 de 0.8323 no conjunto de treinamento.
As previsões foram feitas em 0.0000 segundos.
O modelo calibrado tem F1 de 0.7891 no conjunto de teste.


### Questão 5 - Pontuação F<sub>1</sub> Final
*Qual é a pontuação F<sub>1</sub> do modelo final para treinamento e teste? Como ele se compara ao modelo que não foi calibrado?*

**Resposta: **

A pontuação final foi de 0.8323 no conjunto de treinamento, e de 0.7891 no conjunto de teste. Os valores obtidos foram inferiores aos obtidos para o modelo não calibradom. 

Este resultado inicialmene levantou suspeitas sobre a validade do modelo, porém uma consulta aos escores obtidos para os conjuntos de hiperparâmetros na busca em matriz (vide célula abaico) mostrou que o modelo calibrado realmente apresenta desempenho superior aos demais.

É possível que esta diferença de escore seja devida aos tamanhos muito pequenos de amostra, consequencia do uso de validação cruzada 3-fold durante a busca em matriz, e agravadas pela natureza do dataset (somente 395 observações). Na avaliação são utilizados conjuntos de dados maiores, e essa diferença pode induzir aos desvios [REF: https://stackoverflow.com/questions/49811885/why-is-my-mean-test-score-at-parameter-tuning-cv-lower-than-on-hold-out-test-s]



In [10]:
pd.DataFrame(grid_obj.cv_results_)[['param_C', 'param_penalty', 'rank_test_score', 'mean_test_score', 'mean_train_score']]


C:\Users\cotolino\Anaconda3\envs\py27\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\cotolino\Anaconda3\envs\py27\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\cotolino\Anaconda3\envs\py27\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\cotolino\

,param_C,param_penalty,rank_test_score,mean_test_score,mean_train_score
0,0.01,l1,6,0.811878,0.811883
1,0.01,l2,4,0.813622,0.817667
2,0.05,l1,5,0.812732,0.812689
3,0.05,l2,2,0.822224,0.836541
4,0.1,l1,1,0.830522,0.826288
5,0.1,l2,3,0.818145,0.846665
6,0.3,l1,7,0.808552,0.837455
7,0.3,l2,8,0.802516,0.862568
8,0.5,l1,13,0.796451,0.847769
9,0.5,l2,11,0.798031,0.860693


> **Nota**: Uma vez que você completou todas as implementações de código e respondeu todas as questões acima com êxito, você pode finalizar seu trabalho exportando o iPython Nothebook como um document HTML. Você pode fazer isso utilizando o menu acima e navegando para  
**File -> Download as -> HTML (.html)**. Inclua a documentação final junto com o notebook para o envio do seu projeto.